<a href="https://colab.research.google.com/github/darisoy/EE517_Sp21/blob/master/hw2/BOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm_notebook as tqdm
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

from google.colab import drive
drive.mount('/content/drive')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
class SSTDataset(Dataset):
  def __init__(self, file):
    self.df = pd.read_csv(file, delimiter='\t')
    self.texts = self.df['sentence']
    self.labels = self.df['label']

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index):
    text = self.texts[index]
    label = self.labels[index]

    return (text, label)

In [26]:
tokenizer = get_tokenizer('basic_english')
train_iter = SSTDataset('/content/drive/MyDrive/train.tsv')
counter = Counter()
for i in range(len(train_iter)):
  (line, label) = train_iter[i]
  counter.update(tokenizer(line))
vocab = Vocab(counter, min_freq=1)

In [27]:
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: int(x)

In [28]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_text, _label) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list, text_list, offsets   


In [29]:
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [41]:
num_class = 2
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class)
epochs = 10
LR = 1
BATCH_SIZE = 6
  
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

train_dataloader = DataLoader(SSTDataset('/content/drive/MyDrive/train.tsv'), batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(SSTDataset('/content/drive/MyDrive/dev.tsv'), batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)


In [42]:
# train
for epoch in range(epochs):
  running_loss = 0.0
  for label, text, offsets in tqdm(train_dataloader):
    optimizer.zero_grad()
    outputs = model(text, offsets)
    loss = criterion(outputs, label)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  print('[Epoch %d]\tTrain Loss: \t\t%.3f' % (epoch+1, running_loss / len(train_dataloader)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.



[Epoch 1]	Train Loss: 		0.474



[Epoch 2]	Train Loss: 		0.299



[Epoch 3]	Train Loss: 		0.263



[Epoch 4]	Train Loss: 		0.239


...


[Epoch 6]	Train Loss: 		0.215



[Epoch 7]	Train Loss: 		0.207



[Epoch 8]	Train Loss: 		0.200



[Epoch 9]	Train Loss: 		0.196



[Epoch 10]	Train Loss: 		0.190


In [43]:
#Calculate loss on test set
correct = 0
total = 0
total_loss = 0
with torch.no_grad():
  for label, text, offsets in tqdm(valid_dataloader):
    outputs = model(text, offsets)
    _, predicted = torch.max(outputs.data, 1)
    total += label.size(0)
    correct += (predicted == label).sum().item()
    total_loss += criterion(outputs, label).item()

print('Validation Loss: \t%.3f' % (total_loss / len(valid_dataloader)))
print('Validation Accuracy: \t%.2f %%' % (100 * correct / total))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



Validation Loss: 	0.463
Validation Accuracy: 	80.05 %
